In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [11]:
data = spark.read.csv('/home/ubuntu/Course_Notes/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',inferSchema= True, header = True)

In [12]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [16]:
data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

In [18]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [30]:
assembler = VectorAssembler(inputCols = ['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership'],
                                           outputCol =  'features')

In [31]:
output = assembler.transform(data)

In [32]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [33]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [34]:
final_data = output.select(['features','Yearly Amount Spent'])

In [35]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [36]:
train,test = final_data.randomSplit([0.7,0.3])

In [38]:
train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                363|
|   mean|  498.9788919930656|
| stddev|  79.90508483834039|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [39]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                137|
|   mean| 500.20205354607725|
| stddev|   78.0116822420927|
|    min|  282.4712457199145|
|    max|  712.3963268096637|
+-------+-------------------+



In [40]:
lr = LinearRegression(labelCol= 'Yearly Amount Spent')

In [41]:
lr_model = lr.fit(train)

In [42]:
test_results = lr_model.evaluate(test)

In [46]:
test_results.rootMeanSquaredError

9.911586887569422

In [47]:
test_results.r2

0.9837389435496876

In [48]:
unlabelled_data = test.select('features')

In [49]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6610498227460...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8186165667690...|
+--------------------+
only showing top 20 rows



In [50]:
predicitons = lr_model.transform(unlabelled_data)

In [51]:
predicitons.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 399.7076843200739|
|[30.4925366965402...|287.88121453040253|
|[30.8794843441274...| 494.4866674173911|
|[30.9716756438877...|488.90630511223617|
|[31.1280900496166...| 565.4252808677759|
|[31.1695067987115...|416.87148226499266|
|[31.2834474760581...| 570.5719053614312|
|[31.3091926408918...| 430.3092137631461|
|[31.3895854806643...| 410.4584358909194|
|[31.4459724827577...| 482.4968781652781|
|[31.4474464941278...| 427.6922524178881|
|[31.5147378578019...| 495.9793247728544|
|[31.5261978982398...|418.61064068556516|
|[31.5702008293202...| 564.1131101165595|
|[31.5741380228732...| 559.4398458707369|
|[31.6610498227460...|417.92527202818496|
|[31.7242025238451...|509.82742649031024|
|[31.8093003166791...| 549.1348331567458|
|[31.8164283341993...| 518.6494188399502|
|[31.8186165667690...|449.40111130441255|
+--------------------+------------